# Fall of Sqynet

# Ising Uprising (500 points)

### Backstory

Zenda and Reece model Sqynet as a spin chain, and they come up with a strategy. What if, in addition to using plasma bombs and missiles to increase the temperature of the device, they use a strong magnetic field? After all, magnetic fields might pass through Sqynet's outer shell more easily. The scientists proceed to simulate the effect of a magnetic field on a closed spin chain to quantify the effects.

### Ground state of an Ising spin chain

A simple way to model Sqynet is by considering it as a closed spin chain of length $N$. A spin chain contains particles of spin $1/2$ in each of its
sites. The spins may be pointing in the positive or negative $z$ direction, and we consider that there may be an external magnetic field acting on the system.

![Ising Spin Chain](../img/spaceship_4.png)

Such a quantum system is described by the *Transverse Ising Hamiltonian*. For closed spin chain with a transverse magnetic field of intensity $h$, the Transverse Ising Hamiltonian reads

$$H = -\sum_{i=1}^N Z_i \otimes Z_{i+1} - h \sum_{i=1}^N X_i$$

The subindices $i$ indicate the spin site where the operators act. In a closed spin chain, we identify site $N+1$ with the first site.

A possible plan for Zenda and Reece is to use a strong magnetic field that changes the ground energy of Sqynet, causing it to malfunction.

Your task is to help Zenda and Reece calculate the effect of external magnetic forces on the ground energy. Using the Variational Quantum Eigensolver (VQE) algorithm, you will compute the ground energy of a closed spin chain of length $N=4$.

#### Epilogue

Zenda and Reece fire their powerful magnetic field, missiles, and bombs into the tangle of spins that is Sqynet. As its wavefunction starts to collapse, reality around them shimmers and shifts, and they are enveloped in the purple smoke characteristic of the Oracle World Transform.

The smoke clears. The Bloch clock is ticking; they are late for their weekly brainstorming session at Trine's Designs. Trine is fussing excitedly with the new coffee machine. The network is patchy. Zenda and Reece smile at each other. There are far worse things, they realize, than business as usual.

## Challenge code

In this challenge you will be given the following functions:

- `create_Hamiltonian`: In which you build the Transverse Ising Hamiltonian for $N=4$ and a magnetic field intensity `h`. **You must complete this function**.
- `model`: This QNode builds a general enough ansatz for the ground state. This circuit must depend on some parameters `params`, which you will later optimize. It returns the expectation value of the Hamiltonian for the output state of the circuit. **You must complete this function**.
- `train`: This function returns the parameters that minimize the output of `model`. **You must complete this function**.

### Input

As input to this problem, you are given:

- `h` (`float`): Magnetic field intensity applied to the spin chain.

### Output

This code will output a `float` corresponding to the energy of the ground state.

If your solution matches the correct one within the given tolerance specified in `check` (in this case it's an relative tolerance of `0.1`), the output will be `"Correct!"` Otherwise, you will receive a `"Wrong answer"` prompt.

Good luck!

### Code

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [2]:
def create_Hamiltonian(h):
    """
    Function in charge of generating the Hamiltonian of the statement.

    Args:
        h (float): magnetic field strength

    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """
    
    # Put your code here #
    
    N = 4 # number of spins in chain
    obs = [qml.PauliZ(i) @ qml.PauliZ((i+1)%N) for i in range(N)]
    coeffs = [-1] * N
    obs += [qml.PauliX(i) for i in range(N)]
    coeffs += [-h] * N
    return qml.Hamiltonian(coeffs, obs)
    
create_Hamiltonian(0.1)

  (-0.1) [X0]
+ (-0.1) [X1]
+ (-0.1) [X2]
+ (-0.1) [X3]
+ (-1) [Z0 Z1]
+ (-1) [Z1 Z2]
+ (-1) [Z2 Z3]
+ (-1) [Z3 Z0]


In [3]:
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """
    
    # Put your code here #
    
    n = len(dev.wires)
    
    for i in range(n):
        qml.RX(params[i], wires=i)
    for i in range(n):
        qml.CNOT(wires=[i,(i+1)%n])
    for i in range(n):
        qml.RY(params[i+n], wires=i)
    for i in range(n):
        qml.CNOT(wires=[i,(i+1)%n])
    for i in range(n):
        qml.RZ(params[i+2*n], wires=i)
    
    return qml.expval(H)

print(qml.draw(model)(range(12), create_Hamiltonian(0.1)))
model([1]*12, create_Hamiltonian(0.1))

0: ──RX(0.00)─╭●───────╭X──RY(4.00)─╭●───────╭X──RZ(8.00)──┤ ╭<𝓗>
1: ──RX(1.00)─╰X─╭●────│───RY(5.00)─╰X─╭●────│───RZ(9.00)──┤ ├<𝓗>
2: ──RX(2.00)────╰X─╭●─│───RY(6.00)────╰X─╭●─│───RZ(10.00)─┤ ├<𝓗>
3: ──RX(3.00)───────╰X─╰●──RY(7.00)───────╰X─╰●──RZ(11.00)─┤ ╰<𝓗>


tensor(-0.42358779, requires_grad=True)

In [4]:
def train(h):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.

    Args:
        h (float): magnetic field strength

    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """
    
    # Put your code here #
    
    H = create_Hamiltonian(h)
    params = np.random.rand(12, requires_grad=True)

    # optimising loop
    opt = qml.AdagradOptimizer(stepsize=0.1)
    for _ in range(200): # max iterations
        params = opt.step(lambda x: model(x, H), params)
    
    return params

train(0.1)

tensor([-4.56258180e-02,  2.37991617e-03, -4.14238014e-04,
         1.05232329e-08,  4.62801568e-02,  1.19973697e+00,
         4.65928572e-02,  2.56820802e-03,  1.06368902e-03,
         3.88244917e-14,  1.37325166e+00, -2.70435824e-01], requires_grad=True)

In [5]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = train(ins)
    print(str(model(params, create_Hamiltonian(ins))))
    return str(model(params, create_Hamiltonian(ins)))


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-1
    ), "The expected value is not correct."

In [6]:
test_cases = [['1.0', '-5.226251859505506'], ['2.3', '-9.66382463698038']]

In [7]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.0'...
-4.901622284497634
Correct!
Running test case 1 with input '2.3'...
-9.523950269597668
Correct!
